In [ ]:
# 1. Import Library dan Definisi Konstanta

import pandas as pd

# Nama file Excel utama yang akan dimuat
EXCEL_FILE_NAME = '/content/sample_data/Tracer Study Kemendikbud.xlsx'

# Daftar kolom target standar
TARGET_COLUMNS = [
    'Universitas',
    'program_studi',
    'hor_selaras_jumlah',
    'hor_selaras_pct',
    'hor_tidak_selaras_jumlah',
    'hor_tidak_selaras_pct',
    'mtk_selaras_jumlah',
    'mtk_selaras_pct',
    'mtk_tidak_selaras_jumlah',
    'mtk_tidak_selaras_pct',
    'vert_tinggi_jumlah',
    'vert_tinggi_pct',
    'vert_sama_jumlah',
    'vert_sama_pct',
    'vert_rendah_jumlah',
    'vert_rendah_pct'
]

# Pemetaan kolom Excel (header multi-level) ke nama standar
COLUMN_MAPPING = {
    'Prodi': 'program_studi',
    'Jumlah': 'hor_selaras_jumlah',
    '%': 'hor_selaras_pct',
    'Jumlah.1': 'hor_tidak_selaras_jumlah',
    '%.1': 'hor_tidak_selaras_pct',
    'Jumlah.2': 'mtk_selaras_jumlah',
    '%.2': 'mtk_selaras_pct',
    'Jumlah.3': 'mtk_tidak_selaras_jumlah',
    '%.3': 'mtk_tidak_selaras_pct',
    'Jumlah.4': 'vert_tinggi_jumlah',
    '%.4': 'vert_tinggi_pct',
    'Jumlah.5': 'vert_sama_jumlah',
    '%.5': 'vert_sama_pct',
    'Jumlah.6': 'vert_rendah_jumlah',
    '%.6': 'vert_rendah_pct'
}

In [ ]:
# 2. Fungsi untuk Memproses Satu Sheet

def process_sheet(df: pd.DataFrame, sheet_name: str) -> pd.DataFrame:
    """
    Proses satu DataFrame sheet:
    - Rename kolom sesuai mapping
    - Buat DataFrame kosong dengan kolom TARGET_COLUMNS
    - Isi data, tangani kolom yang hilang
    - Tambahkan kolom 'Universitas'
    - Drop baris sub-header
    """
    # Rename kolom
    df = df.rename(columns=COLUMN_MAPPING, errors='ignore')

    # Inisialisasi DF kosong dengan kolom target
    df_std = pd.DataFrame(columns=TARGET_COLUMNS)

    # Isi setiap kolom
    for col in TARGET_COLUMNS:
        if col in df.columns:
            df_std[col] = df[col]
        else:
            df_std[col] = pd.NA

    # Tambahkan nama universitas
    df_std['Universitas'] = sheet_name.strip()

    # Pastikan urutan kolom sesuai
    df_std = df_std[TARGET_COLUMNS]

    # Hapus baris pertama (sub-header)
    return df_std.iloc[1:].copy()

In [ ]:
# 3. Fungsi Utama : Load Semua Sheet, Proses, dan Gabung

def load_and_combine(excel_path: str) -> pd.DataFrame:
    """
    - Baca semua sheet Excel (header=3)
    - Proses tiap sheet dengan process_sheet
    - Gabungkan menjadi satu DataFrame
    """
    all_sheets = pd.read_excel(excel_path, sheet_name=None, header=3)
    processed = []

    for name, df in all_sheets.items():
        processed.append(process_sheet(df, name))

    combined_df = pd.concat(processed, ignore_index=True)
    return combined_df

In [ ]:
# 4. Simpan ke CSV dan Error Handling

def main():
    try:
        final_df = load_and_combine(EXCEL_FILE_NAME)
        output_csv = 'combined_tracer_study.csv'
        final_df.to_csv(output_csv, index=False)
        print(f"Data berhasil distandardisasi dan disimpan ke {output_csv}")
    except FileNotFoundError:
        print(f"Error: File '{EXCEL_FILE_NAME}' tidak ditemukan.")
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

if __name__ == '__main__':
    main()

Data berhasil distandardisasi dan disimpan ke combined_tracer_study.csv


In [ ]:
# 5. Display Data

final_df = load_and_combine(EXCEL_FILE_NAME)
final_df.head()

,Universitas,program_studi,hor_selaras_jumlah,hor_selaras_pct,hor_tidak_selaras_jumlah,hor_tidak_selaras_pct,mtk_selaras_jumlah,mtk_selaras_pct,mtk_tidak_selaras_jumlah,mtk_tidak_selaras_pct,vert_tinggi_jumlah,vert_tinggi_pct,vert_sama_jumlah,vert_sama_pct,vert_rendah_jumlah,vert_rendah_pct
0,UB,S2 Ilmu Biomedik,79.0,95.18%,4.0,4.82%,31.0,96.88%,1.0,3.13%,15.0,18.07%,62.0,74.7%,6.0,7.23%
1,UB,S1 Kedokteran,843.0,94.4%,50.0,5.6%,25.0,89.29%,3.0,10.71%,475.0,53.19%,406.0,45.46%,12.0,1.34%
2,UB,Sp-1 Ilmu Penyakit Mata,53.0,1,0.0,0,27.0,96.43%,1.0,3.57%,5.0,9.43%,48.0,90.57%,0.0,0
3,UB,Sp-1 Ilmu Penyakit Dalam,79.0,1,0.0,0,27.0,1,0.0,0,10.0,12.66%,65.0,82.28%,4.0,5.06%
4,UB,Sp-1 Ilmu Penyakit Syaraf,34.0,97.14%,1.0,2.86%,23.0,1,0.0,0,2.0,5.71%,33.0,94.29%,0.0,0
